# Ground response on public web data using Google Search 

This notebook shows how to leverage Google Search public data to ground responses generated by Gemini models.

Here are some useful resources that helped building this notebook:

- [Getting Started with Grounding with Gemini in Vertex AI](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb)
- [Ground response on public web data using Google Search](https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/grounding#ground-on-public-data)

## Install libraries

In [ ]:
!pip install --upgrade google-cloud-aiplatform

## Import libraries

In [7]:
from vertexai.preview.generative_models import (
    GenerativeModel,
    GenerationResponse,
    Tool,
    grounding,
)

from IPython.display import display, Markdown

## Choose LLM

In [11]:
model = GenerativeModel("gemini-1.5-pro-preview-0409")

## Helper functions

In [12]:
def print_grounding_response(response: GenerationResponse):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    sources: dict[str, str] = {}
    footnote = 1
    for attribution in grounding_metadata.grounding_attributions:
        context = attribution.web or attribution.retrieved_context
        if not context:
            print(f"Skipping Grounding Attribution {attribution}")
            continue

        title = context.title
        uri = context.uri
        end_index = int(attribution.segment.end_index)

        if uri not in sources:
            sources[uri] = {"title": title, "footnote": footnote}
            footnote += 1

        text_segment = text_bytes[prev_index:end_index].decode(ENCODING)
        markdown_text += f"{text_segment} [[{sources[uri]['footnote']}]]({uri})"
        prev_index = end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    for uri, source in sources.items():
        markdown_text += f"{source['footnote']}. [{source['title']}]({uri})\n"

    display(Markdown(markdown_text))

## Send Prompt

In [13]:
import vertexai
from IPython.display import Markdown

from vertexai.preview.generative_models import grounding
from vertexai.generative_models import GenerationConfig, GenerativeModel, Tool

# TODO(developer): Update and un-comment below line
project_id = "jo-vertex-ai-playground-ffyc"

vertexai.init(project=project_id, location="us-central1")

model = GenerativeModel(model_name="gemini-1.0-pro-002")

# Use Google Search for grounding
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

prompt = "When is the next total solar eclipse in US?"
response = model.generate_content(
    prompt,
    tools=[tool],
    generation_config=GenerationConfig(
        temperature=0.0,
    ),
)

print_grounding_response(response)

#text_response = response.text
#Markdown(text_response)
#print(response)

The next total solar eclipse in the US will be on August 23, 2044. The last total solar eclipse in the US was on April 8, 2024.
## Grounding Sources

**Web Search Queries:** ['When is the next total solar eclipse in the US?']
